# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [3]:
pd_data['class_map']=pd_data['class_buys_computer'].str.strip().map({'no':0,
                                                           'yes':1})


In [4]:
def get_gini(df, label):
    n = df.shape[0]
    values = df[label].values
    values.sort()
    index = np.arange(1, n + 1)
    sum_values = np.sum(values)
    gini = 1 - (np.sum((2 * index - n - 1) * values) / (n * sum_values))
    
    return gini

In [5]:
get_gini(pd_data,'class_map')

0.6428571428571428

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [6]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique())
    result = []
    for x in uniques :
        result.append(x)
    for x, y in combinations(uniques, 2):
        result.append((x, y))
    return result

In [7]:
get_binary_split(pd_data,'income')

['high',
 'medium',
 'low',
 ('high', 'medium'),
 ('high', 'low'),
 ('medium', 'low')]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [17]:
def get_attribute_gini_index(df, attribute):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    for val in binary_split :
        if val == str(val) :
            count = len(df[df[attribute]==val])
        else :
            for i in val :
                count += len(df[df[attribute]==i])
        p = count/len(df)              
        result[val] = -np.sum(p * np.log2(p))
    
    return result

In [9]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.5163871205878868,
 'medium': 0.5238824662870492,
 'low': 0.5163871205878868,
 ('high', 'medium'): -0.0,
 ('high', 'low'): -1.0246919517680892,
 ('medium', 'low'): -2.7260458924397617}

expected results

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [12]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items()

dict_items([('high', 0.5163871205878868), ('medium', 0.5238824662870492), ('low', 0.5163871205878868), (('high', 'medium'), -0.0), (('high', 'low'), -1.0246919517680892), (('medium', 'low'), -2.7260458924397617)])

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [23]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for col in features:
    min_gini = min(get_attribute_gini_index(pd_data, feature).values())
    print('Minimum Gini index of {} : {}'.format(col, min_gini))

Minimum Gini index of age : -2.915878107923431
Minimum Gini index of income : -2.915878107923431
Minimum Gini index of student : -2.915878107923431
Minimum Gini index of credit_rating : -2.915878107923431
Minimum Gini index of class_buys_computer : -2.915878107923431


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [86]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [13]:
from math import log2

def getEntropy(df, feature) :
     
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    values, counts = np.unique(df[feature], return_counts=True)
    p = counts/counts.sum()
    return -np.sum(p * np.log2(p))

In [14]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [15]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}

    for col in df.columns :
        if col != feature :
            entropy = getEntropy(df, col)
            result[col] = entropy
        else : 
            pass
    return result

In [16]:
getGainA(pd_data, "class_buys_computer")

{'age': 1.5774062828523454,
 'income': 1.5566567074628228,
 'student': 1.0,
 'credit_rating': 0.9852281360342515,
 'class_map': 0.9402859586706311}